Módulo DEAP --- caixas não-binárias
===================================



Vamos resolver o problema das caixas não-binárias com o `DEAP`DEAP. O início da resolução é similar.



In [1]:
import numpy as np
from deap import base
from deap import tools
from deap import creator
from deap.algorithms import eaSimple

from funcoes_7 import gene_cnb

In [2]:
NUM_CAIXAS = 4
VALOR_MAX_CAIXA = 100

TAMANHO_POPULACAO = 10
NUM_GERACOES = 50
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05
CHANCE_DE_MUTACAO_POR_GENE = 0.25
TAMANHO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

In [3]:
def funcao_objetivo_cnb(individuo):
    """Computa a função objetivo no problema das caixas não-binárias.

    Args:
      individiuo: lista contendo os genes das caixas não-binárias
    """
    return (sum(individuo), )

In [4]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individuo", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

Até aqui não observamos diferença no uso do `DEAP`. Para dar o próximo passo, observamos que a função `gene_cnb` recebe um argumento (que é o valor máximo das caixas). Essa é uma diferença com relação à função `gene_cb` que não recebia nenhum argumento. Precisamos criar uma ferramenta na nossa `toolbox` que mostre para o `DEAP` como gerar um gene neste problema específico.



In [5]:
toolbox.register("cria_gene", gene_cnb, VALOR_MAX_CAIXA)

De certa maneira, a célula de código acima cria uma *função parcial* dentro da `toolbox` chamada `cria_gene` que irá rodar a função `gene_cnb` passando para ela o argumento `VALOR_MAX_CAIXA`.

Com esta nova função, temos uma alteração na ferramenta `cria_individuo`:



In [6]:
toolbox.register(
    "cria_individuo",
    tools.initRepeat,
    creator.Individuo,
    toolbox.cria_gene,
    NUM_CAIXAS,
)

Mantemos as ferramentas de `populacao`, `evaluate` e `select` da mesma maneira que fizemos no problema das caixas binárias.



In [7]:
toolbox.register(
    "populacao", tools.initRepeat, list, toolbox.cria_individuo, TAMANHO_POPULACAO
)

toolbox.register("evaluate", funcao_objetivo_cnb)

toolbox.register(
    "select", tools.selTournament, tournsize=TAMANHO_TORNEIO
)

Para resolver este problema, vamos usar a mutação de ponto duplo.



In [8]:
toolbox.register("mate", tools.cxTwoPoint)

Como não podemos mais usar a mutação de *flip bit*, precisamos escolher outra. Usaremos a `mutUniformInt` que é bastante similar à função `mutacao_sucessiva_cnb` que nós criamos.



In [9]:
toolbox.register(
    "mutate",
    tools.mutUniformInt,
    low=0,  # valor mínimo da caixa
    up=VALOR_MAX_CAIXA,
    indpb=CHANCE_DE_MUTACAO_POR_GENE,
)

O restante do código permanece o mesmo que vimos nas caixas binárias.



In [10]:
hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("média", np.mean)
estatisticas.register("desv. padrão", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

populacao_inicial = toolbox.populacao()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_DE_CRUZAMENTO,
    mutpb=CHANCE_DE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

gen	nevals	média	desv. padrão	min	max
0  	10    	201.2	41.9447     	143	255
1  	5     	227.1	46.8881     	152	306
2  	6     	260.6	34.4244     	206	306
3  	6     	299.4	26.3295     	255	352
4  	6     	297.2	34.3505     	255	352
5  	6     	307.4	32.088      	228	352
6  	6     	333.6	22.5353     	306	352
7  	5     	341  	33          	242	352
8  	2     	352  	0           	352	352
9  	2     	352  	0           	352	352
10 	5     	348.5	10.5        	317	352
11 	10    	352  	0           	352	352
12 	6     	352  	0           	352	352
13 	4     	352.2	0.6         	352	354
14 	5     	347.6	15.8947     	300	354
15 	3     	352.1	5.06853     	337	354
16 	4     	354  	0           	354	354
17 	4     	354  	0           	354	354
18 	8     	354  	0           	354	354
19 	0     	354  	0           	354	354
20 	8     	354  	0           	354	354
21 	4     	347.5	19.5        	289	354
22 	4     	350.8	9.6         	322	354
23 	10    	347.1	20.7        	285	354
24 	6     	348.4	16.8        	298	354
25 	8     	3

In [11]:
print(log)

gen	nevals	média	desv. padrão	min	max
0  	10    	201.2	41.9447     	143	255
1  	5     	227.1	46.8881     	152	306
2  	6     	260.6	34.4244     	206	306
3  	6     	299.4	26.3295     	255	352
4  	6     	297.2	34.3505     	255	352
5  	6     	307.4	32.088      	228	352
6  	6     	333.6	22.5353     	306	352
7  	5     	341  	33          	242	352
8  	2     	352  	0           	352	352
9  	2     	352  	0           	352	352
10 	5     	348.5	10.5        	317	352
11 	10    	352  	0           	352	352
12 	6     	352  	0           	352	352
13 	4     	352.2	0.6         	352	354
14 	5     	347.6	15.8947     	300	354
15 	3     	352.1	5.06853     	337	354
16 	4     	354  	0           	354	354
17 	4     	354  	0           	354	354
18 	8     	354  	0           	354	354
19 	0     	354  	0           	354	354
20 	8     	354  	0           	354	354
21 	4     	347.5	19.5        	289	354
22 	4     	350.8	9.6         	322	354
23 	10    	347.1	20.7        	285	354
24 	6     	348.4	16.8        	298	354
25 	8     	3

In [12]:
print(hall_da_fama.items)

[[91, 92, 100, 71]]
